In [ ]:
import pandas as pd
import numpy as np
import os

path = 'D:\\maran10\\Documents\\日常工作\\6.数据复盘\\'
path_goldeye = path+'1.2020H1黄金眼\\原数据\\'
path_goldeye_new = path+'1.2020H1黄金眼\\加上行业后\\'
# path_goods = path+'全sku商品池\\c_商用订单明细_（2019H1和2020H1）\\年份品类拆分\\'

In [ ]:
pre = pd.read_excel(path+'ep客户.xlsx',encoding='gbk')
pre['企业-销售'] = pre['企业名称系统存储'].apply(lambda x:str(x))+pre['saler_erp'].apply(lambda x:str(x))
pre.to_excel(path+'处理后_EP客户.xlsx',encoding='gbk',index=False)

In [ ]:
lst = os.listdir(path_goldeye)    #列出文件夹下所有的目录与文件
print(lst)

for i in range(0,len(lst)):
    pth = os.path.join(path_goldeye,lst[i])
    if os.path.isdir(pth):
        print("当前路径",pth)
        p = os.listdir(pth)
#         print("当前路径下文件",p)
        count = 0
        for f in p:
            if not ".xlsx" in f:
                continue
            order = pd.read_excel(pth+"\\"+f)
            order = order.drop_duplicates(subset=None,keep='first')
            print(f,"去重后订单总量",len(order))
            
            order['企业-销售'] = order['企业名称'].apply(lambda x:str(x))+order['销售员erp'].apply(lambda x:str(x))
            order_new = pd.merge(order,pre[['企业-销售','corp_stats_name_1','corp_stats_name_2']],on='企业-销售',how='left')
            
            order_new.to_excel(path_goldeye_new+"加行业后_"+f,index=False,encoding='gbk')


In [ ]:
def grouped_sum(df,group_col,sum_col):
        df_join = df.groupby(group_col)['订单金额'].sum()
        df_join = pd.DataFrame(df_join)
        df_join.columns = [sum_col]
        df_join.reset_index(group_col,inplace=True)
        return df_join


In [ ]:
lst_new = os.listdir(path_goldeye_new)    #列出文件夹下所有的目录与文件
print(lst_new)

for i in range(0,len(lst_new)):
    pth_new = os.path.join(path_goldeye_new,lst_new[i])
    if os.path.isdir(pth_new):
        print("当前路径",pth_new)
        
        #### 要把这个加总放在对应品类的文件夹下
        gold_dict = {}
        gold_dict_joincom = {}
        gold_dict_joinind = {}
        gold_dict_sumind = {}
        gold_dict_countind = {}
        comp_dict = {}

        new_goldeye = os.listdir(pth_new)
        for i,fnew in enumerate(new_goldeye):
            if not ".xlsx" in fnew:
                continue
            gold_dict[i] = pd.read_excel(pth_new+"\\"+fnew,usecols=['一级部门','三级分类','品牌','区域','企业名称','销售员erp','企业-销售','corp_stats_name_1','订单金额'])
            gold_dict[i] = gold_dict[i].fillna('空白')
            
            gold_dict_joincom[i] = grouped_sum(gold_dict[i],'企业名称','H1订单金额求和（按客户）')
            print(i,fnew,"加总后企业-销售额数据条数",len(gold_dict_joincom[i]))  

            if i == 0:
                new_com = pd.DataFrame(columns = gold_dict_joincom[i].columns)
            new_com = pd.concat([new_com,gold_dict_joincom[i]])
            print(i,'拼接后客户-销量数据条数',len(new_com))

        new_com_join = new_com.groupby('企业名称')['H1订单金额求和（按客户）'].sum()
        new_com_join = pd.DataFrame(new_com_join)
        new_com_join.columns = ['H1订单金额求和']
        new_com_join.reset_index('企业名称',inplace=True)
        new_com_join.to_excel(pth_new+'\\处理\\该品类企业名称-订单金额求和_新.xlsx',index=False)           
            

            if i == 0:
                for ind in gold_dict[i]['三级分类'].unique():
                    comp_dict[ind] = set(gold_dict[i][gold_dict[i]['三级分类']==ind]['企业名称'])
            else:
                for ind in gold_dict[i]['三级分类'].unique():
                    temp_set = set(gold_dict[i][gold_dict[i]['三级分类']==ind]['企业名称'])
                    if ind not in comp_dict.keys():
                        comp_dict[ind] = temp_set
                    else:
                        comp_dict[ind] = comp_dict[ind].union(temp_set)            

           
            gold_dict_sumind[i] = grouped_sum(gold_dict[i],'三级分类','H1订单金额求和（按三级分类）')
#             gold_dict_countind[i] = grouped_count(gold_dict[i],'corp_stats_name_1','客户数量')
#             gold_dict_joinind[i] = pd.merge(gold_dict_sumind[i],gold_dict_countind[i],how='outer')
            print(i,fnew,"加总后三级分类-销售额数据条数",len(gold_dict_sumind[i]))
            

            if i == 0:
                new_ind = pd.DataFrame(columns = gold_dict_sumind[i].columns)
            new_ind = pd.concat([new_ind,gold_dict_sumind[i]])
            print(i,'拼接后三级分类-销售额数据条数',len(new_ind))

        new_ind_sum = new_ind.groupby(['三级分类'])['H1订单金额求和（按三级分类）'].sum()
        new_ind_sum = pd.DataFrame(new_ind_sum)
        new_ind_sum.columns = ['H1订单金额求和']
        
        max_length = 0
        for k in comp_dict.keys():
            if max_length < len(comp_dict[k]):
                max_length = len(comp_dict[k])
        for k in comp_dict.keys():
            comp_dict[k] = list(comp_dict[k])+[np.nan] * (max_length - len(comp_dict[k]))

        comp_df = pd.DataFrame(comp_dict)
        comp_df.to_excel(pth_new+'\\处理\\该品类三级分类_客户名单.xlsx',index=False)
        print(pth_new,'该品类各三级分类客户名单已储存')

        for k in comp_dict.keys():
            new_ind_sum.loc[k, '客户数量'] =  comp_df[k].count()

        new_ind_sum.reset_index('三级分类',inplace=True)
        new_ind_sum.to_excel(pth_new+'\\处理\\该品类三级分类-订单金额求和-客户去重计数.xlsx',index=False)
        
